Updates to script for version 2

1.   Using PySpark & Pandas for better performance & speed
2.   Using Azure Cousmos DB to send cleaned data to PowerBI Dashboard
3.   Minor usage of Numba Framework to speed up some methods within this script
4.   Potential Trials of Time-Series Machine Learning Modeling of Data 
5.   Trials with using the Libra Library for potential simple Machine Learning development
6.   Determine if using Azure Synapse makes sense for this project for giving no-code manipulation of the data

Website Links & Documentation for all program enhancements

1.   PySpark Examples - https://github.com/apache/spark/tree/f74867bddf/examples/src/main/python
2.   Azure Cosmos DB - https://docs.microsoft.com/en-us/azure/cosmos-db/sql/create-sql-api-python
2.   Azure Cosmos DB Github Demo - https://github.com/Azure-Samples/azure-cosmos-db-python-getting-started/blob/main/cosmos_get_started.py
3.   Numba - https://numba.pydata.org/
4.   Time-Series Machine Learning Algos - https://www.advancinganalytics.co.uk/blog/2021/06/22/10-incredibly-useful-time-series-forecasting-algorithms
5.   Libra - https://libradocs.org/
6.   Azure Synapse & Cosmos DB Integration - https://docs.microsoft.com/en-us/azure/cosmos-db/synapse-link
7.   Spark SQL & Python - https://spark.apache.org/docs/latest/sql-programming-guide.html

In [1]:
#Start by importing the data from google sheet
%pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [2]:
#all imports 
import gspread

In [ ]:
#